во всех формулах из лекции до ap_k() есть смысловая ошибка
```
flags = np.isin(bought_list, recommended_list)
```
должно быть
```   
flags = np.isin(recommended_list, bought_list)
```


In [95]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)

    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list)    
    
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    return hit_rate_at_k
    
    

In [96]:
import numpy as np
recommended_list = [156, 143, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]


In [97]:
hit_rate_at_k(recommended_list, bought_list, k=4)

1

In [98]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended  = np.array(prices_recommended[:k])

    flags = np.isin(recommended_list, bought_list)

    money_precision_at_k = flags.dot(prices_recommended.T).sum() / prices_recommended.sum()
    
    return money_precision_at_k

In [99]:
bought_list = [502, 512 , 1, 991, 2]
recommended_list = [51, 512, 75, 99, 991, 7, 1, 432]
prices_recommended = [400, 60, 40, 40, 90 , 10 , 40, 60]

In [100]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=7)

0.27941176470588236

In [101]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list)
    recall_at_k = flags.sum() / len(bought_list)
    
    return recall_at_k


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended  = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
    
    money_recall_at_k = flags.dot(prices_recommended) / bought_list.dot(prices_bought.T)
    
    
    return money_recall_at_k

In [102]:
recall(recommended_list, bought_list)

0.6

In [103]:
prices_bought = [10, 50, 100, 200 ,300]

In [104]:
for i in range(1, 9):
    print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=i))

0.0
0.00026141512722202857
0.00026141512722202857
0.00026141512722202857
0.0006535378180550714
0.0006535378180550714
0.0008278145695364238
0.0008278145695364238


в лекци ошибка - функция ap_k не может посчитать k = "длинна списка рекомендаций' из-за:
```
for i in range(1, k+1):
        
        if flags[i] == True:
```
тут будет перебор индекса в этом случае, так что я подправил формулу что бы она считала корректно.


In [105]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k):
        
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [166]:
users_purchases = {
    'user1': {
        'price_bought' : [10, 50, 100, 200 ,300],
        'bought_list' : [502, 512 , 1, 991, 2],
        'recommended_list' : [51, 512, 75, 99, 991, 7, 1, 432]

    },
    'user2': {
        'price_bought' : [10, 50, 100, 200 ,300],
        'bought_list' : [502, 13 , 99, 991, 2],
        'recommended_list' : [51, 13, 75, 99, 991, 7, 1, 432]
    },
    'user3': {
        'price_bought' : [10, 50, 100, 200 ,300],
        'bought_list' : [502, 13 , 99, 991, 2],
        'recommended_list' : [1, 1, 1, 1, 991, 7, 1, 1]
    }
 }

In [167]:
def map_k(users_purchases, k=5):
        
    sum_ = 0
    for user in users_purchases:
        sum_ += ap_k(
            users_purchases[user]['recommended_list'],
            users_purchases[user]['bought_list'],
            k=k
            )

    result = sum_ / len(users_purchases)

    return result

In [168]:
for i in range(1,9):
    print(map_k(users_purchases=users_purchases, k=i))

0.0
0.1111111111111111
0.1111111111111111
0.16666666666666666
0.3444444444444444
0.3444444444444444
0.39206349206349206
0.39206349206349206


In [169]:
ap_k(recommended_list, bought_list, k=8)

0.44285714285714284

In [170]:
def reciprocal_rank(users_purchases, k=5):
    result = []

    for user in users_purchases:

        recommended_list = np.array(users_purchases[user]['recommended_list'][:5])
        bought_list = np.array(users_purchases[user]['bought_list'])
        
        flags = np.isin(recommended_list, bought_list)

        if flags.any():
            result.append(1 / (1+flags.nonzero()[0][0]))    
        else:
            result.append(0)
    return sum(result)/len(result)

In [171]:
reciprocal_rank(users_purchases)

0.39999999999999997